<a href="https://colab.research.google.com/github/monesh1981/GEE_Projectmkb/blob/main/Assignment_Bihar_LULC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importing Required Libraries**

1. geemap: A Python package used for interactive visualization and analysis of geospatial data, particularly with Google Earth Engine (GEE).
2. ee: The Earth Engine Python API, which allows you to interact with Google's cloud-based geospatial analysis platform.

In [252]:
import geemap
import ee

**Authenticating with Google Earth Engine**

This function prompts the user to authenticate their Google Earth Engine (GEE) account.
It usually requires a Google account and may open a web browser for login or provide a link to manually authenticate.

In [253]:
ee.Authenticate()


True

**Initializing Earth Engine with a Project**

 Initializes the Earth Engine API to start using its functionalities. The project="ee-moneshbathre" specifies a GEE project associated with my Google Cloud account.

In [254]:
ee.Initialize(project = "ee-moneshbathre")


**Creating a Map for Visualization**

geemap.Map() creates an interactive map object.
This map can be used to visualize geospatial data from GEE, including satellite imagery, vector datasets, and raster layers.

mapviz is the variable.

In [255]:
mapviz = geemap.Map()

**Defining the Study Area**

ee.FeatureCollection(...) loads a vector dataset from  Earth Engine (EE) assets.

'projects/ee-moneshbathre/assets/bihar_bdy' is the path to the asset in my EE account. This asset represents the boundary of Bihar, which is a state in India.

In [257]:
study_area = ee.FeatureCollection('projects/ee-moneshbathre/assets/bihar_bdy')



Map(center=[25.903891043806432, 85.80700650480884], controls=(WidgetControl(options=['position', 'transparent_…

**Adding the Study Area to the Map**

layer = study_area: Adds the Bihar boundary to the interactive map.

vis_params = {}: Empty visualization parameters (default style).

'Bihar': The layer name shown in the map legend.

Centering the Map on Bihar and Displaying it.

In [ ]:
mapviz.addLayer(study_area,{},'Bihar')
mapviz.centerObject(study_area)
mapviz

#adding legend and visualisation parameters for lulc as per ESA worldcover

**value_visual** is a dictionary that stores visualization settings for a raster dataset in Google Earth Engine (GEE).

**Keys:**
'min': 10 → Minimum data value mapped to the color scale.
'max': 100 → Maximum data value mapped to the color scale.
'palette': A list of color codes (hex values) representing different land cover types.

**Legend** is a dictionary mapping land cover types to their corresponding color codes.
This helps in interpreting the colors used in the value_visual palette.

In [242]:
value_visual = {'min':10,'max':100,'palette':['#006400','#ffbb22','#ffff4c','#f096ff','#fa0000','#b4b4b4','#f0f0f0','#0064c8','#0096a0','	#00cf75','#fae6a0']}
Legend = {'Tree Cover':'#006400',
          'Shrubland':'#ffbb22',
          'Grassland':'#ffff4c',
          'Cropland':'#f096ff',
          'Built-up':'#fa0000',
          'Bare / sparse vegetation':'#b4b4b4',
          'Snow and ice':'#f0f0f0',
          'Permanent water bodies':'#0064c8',
          'Herbaceous wetland':'#0096a0',
          'Mangroves':'#00cf75',
          'Moss and lichen':'#fae6a0'}

Clip ESA World Cover Data limited to Chattisgarh studay area

Loads the ESA WorldCover v200 dataset, which provides global land cover data.
This dataset is hosted on Google Earth Engine (GEE).
It contains different land cover types for the entire world.

**Computing the Mean Image**
The dataset is an ImageCollection (a stack of images over time).
.mean() computes the average of all images in the collection.
This is useful if multiple time-periods are available in the dataset.

 **Clipping the Image to the Study Area**
 clip(study_area) trims the image to the boundaries of Bihar (your study area).
This ensures that only Bihar’s land cover is shown on the map.

**Selecting the Land Cover Band**
The dataset contains multiple bands.
select('Map') extracts the "Map" band, which represents land cover classification.

**Adding the Layer to the Map**

cg_esa: The clipped land cover image for Bihar.

value_visual: The visualization parameters (min, max, palette).

'Bihar ESA Worldcover': The layer name in the interactive map.



In [243]:
cg_esa = ee.ImageCollection('ESA/WorldCover/v200').mean().clip(study_area).select('Map')
mapviz.addLayer(cg_esa,value_visual,'Bihar ESA Worldcover')


**Add a legend to your interactive map**

title of the legend as "My Legend".

The Legend dictionary (defined earlier) maps land cover types to their corresponding color codes.

This helps users understand the meaning of different colors in the Bihar land cover map.

In [258]:
mapviz.add_legend(title="My Legend", legend_dict=Legend, **value_visual)

**Clip Landsat 8 data for study area and dates/cloudcover**

 Load Landsat 8 Data for 2024

Filter images with less than 5% cloud cover

Average the filtered images for a clean view

Clip the image to Bihar’s boundary

 Add the layer to an interactive map (mapviz)


In [245]:
landsat8_data = ee.ImageCollection("LANDSAT/LC08/C02/T1").filterDate('2024-01-01', '2024-12-31').filterMetadata('CLOUD_COVER','less_than',5).mean().clip(study_area)
mapviz.addLayer(landsat8_data,{},'Landsat8_2024')
mapviz

Map(bottom=7366.0, center=[25.903891043806432, 85.80700650480884], controls=(WidgetControl(options=['position'…

**Adding Training Points**

Creates a sample of points from the ESA WorldCover dataset.

region=study_area.geometry() → Restricts sampling to Bihar’s boundary.

scale=20 → Sets the spatial resolution to 20 meters per pixel.

numPixels=4000 → Extracts 4,000 random points.

seed=1 → Ensures reproducibility (same random points every time).

geometries=True → Keeps the points as vector geometries (not just numeric values).

'Training Points' will appear as a layer in the interactive map.




In [260]:
TP = cg_esa.sample(**{
    'region':study_area.geometry(),
    'scale':20,
    'numPixels':4000,
    'seed':1,
    'geometries':True
    })
mapviz.addLayer(TP,{},'Training Points')
mapviz



Map(bottom=7435.0, center=[22.61401087437029, 89.22796734701859], controls=(WidgetControl(options=['position',…

**Performing supervised land cover classification using Landsat 8 imagery and ESA WorldCover training points in Google Earth Engine**

**Selecting Landsat 8 Bands for Classification.** This defines the input bands (spectral bands from Landsat 8).
These bands are used to train the classification model.

**Defining the Label for Classification**. 'Map' is the label column in your training dataset (TP).
This column contains the land cover class from the ESA WorldCover dataset.
The classifier will learn to predict this land cover class based on Landsat 8 bands.


** Sampling Training Data**

Extracts Landsat 8 spectral values at the training point locations (TP).
properties=[label] ensures that land cover labels are included.
scale=30 matches Landsat 8’s spatial resolution (30 meters per pixel).
Output: training dataset now contains:
Spectral values from Landsat 8 bands (B1 to B6).
Land cover labels from ESA WorldCover (Map column).

**Training the Classifier:** Trains a k-Nearest Neighbors (KNN) classifier using the training data.

Inputs:

**training** → The dataset with Landsat bands + land cover labels.

label → The target variable (Map column).

bands → The input features (Landsat bands).

**Classifying the Image:** Uses the trained KNN classifier to classify every pixel in the Landsat 8 image.
Output: result is a classified land cover map.




In [259]:
bands = ['B1','B2','B3','B4','B5','B6']
label = 'Map'
training = landsat8_data.select(bands).sampleRegions(**{'collection':TP,
                                       'properties':[label],
                                       'scale':30})
trained = ee.Classifier.smileKNN().train(training,label,bands)
result = landsat8_data.select(bands).classify(trained)

**Adding the Classified Image to the Map**

This line adds the classified land cover map (result) to our interactive map (mapviz).
The value_visual dictionary we defined earlier is used to apply the color palette to the classified map.

In [261]:
mapviz.addLayer(result,value_visual,'Supervised_classification')
mapviz

Map(bottom=28840.0, center=[22.61401087437029, 89.22796734701859], controls=(WidgetControl(options=['position'…